In [8]:
import cv2 as cv
from matplotlib import pyplot as plt
import numpy
import os

In [9]:
print(cv.__version__)
ImageFilePath = "/home/mitski/Market-1501-v15.09.15/bounding_box_train/"
NetFilePath = "/home/mitski/model/youtu_reid_baseline_lite.onnx"
BatchSize = 32;

4.5.1


In [10]:
def showImage(img):
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    plt.imshow(img)

In [18]:
def normalizeMat(img): #need implementation on CUDA or GPU (if not support CUDA)
    mean = [ 0.485, 0.456, 0.406 ]
    std = [ 0.229, 0.224, 0.225 ]
    imgRow, imgCols, imgDim = numpy.shape(img)
    ret = numpy.zeros((imgRow,imgCols,imgDim), numpy.float32)
    for y in range(0, imgRow):
        for x in range(0, imgCols):
            for c in range(0, imgDim):
                ret[y,x,c] = (img[y,x,c] / 255 - mean[2-c]) / std[2-c]
    return ret

In [22]:
def extractFeatures(model, batch):
    blob = cv.dnn.blobFromImages(batch, 1.0, (128,256), (0.0, 0.0, 0.0), True, False, cv.CV_32F)
    model.setInput(blob)
    e1 = cv.getTickCount();
    out = model.forward();
    e2 = cv.getTickCount();
    time = (e2 - e1) / cv.getTickFrequency();
    print("extract time " + str(time))
    print(numpy.shape(out))
    return out

In [25]:
model = cv.dnn.readNet(NetFilePath)
model.setPreferableBackend(cv.dnn.DNN_BACKEND_CUDA)
model.setPreferableTarget(cv.dnn.DNN_TARGET_CUDA)

In [31]:
classifierSVM = cv.ml.SVM_create();
classifierSVM.setType(cv.ml.SVM_C_SVC);
classifierSVM.setKernel(cv.ml.SVM_RBF);
classifierSVM.setTermCriteria((cv.TERM_CRITERIA_MAX_ITER, 10000, 1e-5));

In [ ]:
count = 0
images = []
features = []
for filename in os.listdir(ImageFilePath):
    if (len(images) != 5):
        if (count % 100 == 0):
            e1 = cv.getTickCount();
            out = normalizeMat(cv.imread(ImageFilePath + filename));
            e2 = cv.getTickCount();
            time = (e2 - e1) / cv.getTickFrequency();
            print(str(count) + " " + str(time))
            images.append(out)
    else:
        features.append(extractFeatures(model, images))
        images.clear()
    
    count+=1

In [87]:
img = cv.imread(ImageFilePath + "0002_c1s1_000451_03.jpg")

In [91]:
imgNorm = normalizeMat(img)

In [106]:
blob = cv.dnn.blobFromImage(imgNorm, 1.0, (128,256), (0.0, 0.0, 0.0), True, False, cv.CV_32F)
model.setInput(blob)
e1 = cv.getTickCount();
out = model.forward();
e2 = cv.getTickCount();
time = (e2 - e1) / cv.getTickFrequency();

(1, 768, 1, 1)
0.075332163
